In [23]:
#installing nltk for processing
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached regex-2023.3.23-cp39-cp39-macosx_10_9_x86_64.whl (294 kB)
You should consider upgrading via the '/Users/danielcrake/Desktop/Masters/Year 2/Information Retrieval /Assignment/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import CountVectorizer

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import re

from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer  # lemmatization
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer



In [3]:
data = pd.read_csv('metadata.tsv', sep = '\t')
data = data.loc[:,['episode_uri','show_name', 'show_description', 'episode_name', 'episode_description']]
data['docno'] = data.index
data = data.astype(str)
data.head()

,episode_uri,show_name,show_description,episode_name,episode_description,docno
0,spotify:episode:000A9sRBYdVh66csG2qEdj,Kream in your Koffee,A 20-something blunt female takes on the world...,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,0
1,spotify:episode:000HP8n3hNIfglT2wSI2cA,Morning Cup Of Murder,Ever wonder what murder took place on today in...,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",1
2,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,2
3,spotify:episode:001i89SvIQgDuuyC53hfBm,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,3
4,spotify:episode:0025RWNwe2lnp6HcnfzwzG,FBoL,"The comedy podcast about toxic characters, wri...","The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,4


In [4]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# pre-processing
def pre_process(text):
    punctuation_marks = string.punctuation.replace("'", "")

    # separate punctuation from words - preserving apostrophe
    if preprocessing_switches['separate_out_punctuation']:
        for c in punctuation_marks:
            text = text.replace(c, ' ' + c + ' ')

    #converting numbers to digits
    if preprocessing_switches["convert_number_words_to_digits"]:
        text = text2int(text)   

    #removing numbers
    if preprocessing_switches["convert_numbers"]:
        text = re.sub('\d+', 'NUMBER',text)
        text = text.replace('NUMBER ', '')

        #converts to lower case
    if preprocessing_switches['convert_to_lowercase']:    
        text = text.lower()

    # Tokenize the text
    tokens = text.split()

    # Remove stopwords
    if preprocessing_switches['separate_out_punctuation']:    
        stop_words = set(stopwords.words("english"))
        tokens = [token for token in tokens if token not in stop_words]

    # Remove punctuation
    if preprocessing_switches['remove_punctuation']:    
        tokens = [token for token in tokens if token not in punctuation_marks]

    # Lemmatize the tokens
    if preprocessing_switches['apply_lemmatization']:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stem the tokens
    if preprocessing_switches['stem_tokens']:
        tokens = [stemmer.stem(token) for token in tokens]
            

    return ' '.join(tokens)

In [5]:
preprocessing_switches = {'convert_to_lowercase': True,
                            'separate_out_punctuation': True,
                            'remove_punctuation': True,
                            'convert_number_words_to_digits': True,
                            'convert_numbers': True,
                            'remove_stopwords': True,
                            'apply_lemmatization': False,
                            'stem_tokens': False}


In [6]:
def text2int(textnum, numwords={}):
    if not numwords:
        units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]

        tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

        scales = ["hundred", "thousand", "million", "billion", "trillion"]

        numwords["and"] = (1, 0)
        for idx, word in enumerate(units):  numwords[word] = (1, idx)
        for idx, word in enumerate(tens):       numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]

    textnum = textnum.replace('-', ' ')

    current = result = 0
    curstring = ""
    onnumber = False
    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if word not in numwords:
                if onnumber:
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
            else:
                scale, increment = numwords[word]

                current = current * scale + increment
                if scale > 100:
                    result += current
                    current = 0
                onnumber = True

    if onnumber:
        curstring += repr(result + current)

    return curstring

In [8]:
#create subset for processing and apply pre-processing
data_subset = data.loc[:10000]

data_processed = data_subset['show_description'].apply(pre_process).to_list()

In [9]:
vectorizer = CountVectorizer()
documents_vectorized = vectorizer.fit_transform(data_processed)
vocabulary = vectorizer.get_feature_names_out()

In [10]:
dataframe = pd.DataFrame(documents_vectorized.toarray(), columns=vocabulary)

# removing low word counts to improve performance - low word counts also not likely to be relevant
word_counts = dataframe.sum(axis=0)
to_remove = word_counts[word_counts <4].index
dataframe = dataframe.drop(columns = to_remove)
dataframe.shape

(10001, 7266)

In [11]:
#term frequencies
dfs = (dataframe > 0).sum(axis=0)
#idfs
N = dataframe.shape[0]
idfs = np.log10(N/dfs)
#BM25
k_1 = 1.2
b = 0.8

# The following line of code considers all words in each document:
dls_all = [len(d.split(' ')) for d in data_processed] # vector

# But we're not intrested in stop words, therefore, let's do the following: 
dls = dataframe.sum(axis=1).tolist()  # document lengths in words
avgdl = np.mean(dls) # avarage document length

numerator = np.array((k_1 + 1) * dataframe)
denominator = np.array(k_1 *((1 - b) + b * (dls / avgdl))).reshape(N, 1) + \
                       np.array(dataframe)

BM25_tf = numerator / denominator

idfs = np.array(idfs)  # inverse document frequencies

BM25_score = idfs * BM25_tf

In [12]:
bm25_idf = pd.DataFrame(BM25_score, columns=dataframe.columns)
bm25_idf['christmas'].sort_values(ascending = False)

7392     4.806932
2077     3.859564
6765     3.859564
9530     3.859564
2084     3.722615
           ...   
3335     0.000000
3336     0.000000
3337     0.000000
3338     0.000000
10000    0.000000
Name: christmas, Length: 10001, dtype: float64

In [13]:
data.loc[7392]

episode_uri                       spotify:episode:0XgrKcGfkaf3cwGxkwKe5e
show_name                                           61 Days of Christmas
show_description       Hosts Abby and Kelsy watch the 40 new Hallmark...
episode_name             Christmas Wishes & Mistletoe Kisses Snoozefest 
episode_description    Hosts Abby and Kelsy recount the exhausting fi...
docno                                                               7392
Name: 7392, dtype: object

In [95]:
query = 'christmas'
columns = ['show_name', 'show_description']

query_words = pre_process(query).split(' ')
top_results = bm25_idf[query_words].sum(axis = 1).sort_values(ascending = False).index[:5].tolist()
data.loc[top_results, columns]

,show_name,show_description
7392,61 Days of Christmas,Hosts Abby and Kelsy watch the 40 new Hallmark Christmas movies so you don’t have to.
2077,A Cup Of Cheer: A Seasonal Holiday Podcast,"We don’t know if there will be snow, so have a cup of cheer... Hi All, Welcome to A Cup Of Cheer. Here you will find a Christmas Variety Show with many things to offer, movie reviews, recipes, s..."
6765,A Cup Of Cheer: A Seasonal Holiday Podcast,"We don’t know if there will be snow, so have a cup of cheer... Hi All, Welcome to A Cup Of Cheer. Here you will find a Christmas Variety Show with many things to offer, movie reviews, recipes, s..."
9530,A Cup Of Cheer: A Seasonal Holiday Podcast,"We don’t know if there will be snow, so have a cup of cheer... Hi All, Welcome to A Cup Of Cheer. Here you will find a Christmas Variety Show with many things to offer, movie reviews, recipes, s..."
2084,MC Crue Podcast,"On this podcast, we'll be talking about anything and everything, especially what makes a movie a Christmas movie! You can follow the guys on Instagram @mc.crue.podcast or e-mail us segment ideas..."


In [16]:
# from sqlalchemy import create_engine
# engine = create_engine('sqlite:///IRDB', echo=False)
# bm25_idf.to_sql('users', con=engine)

(bm25_idf.memory_usage(deep = True).sum()) / (1024 * 1024)
# bm25_idf.to_csv('Web Page/BM25_data.csv')

554.4073638916016

In [17]:
data.loc[2084, 'show_description']

"On this podcast, we'll be talking about anything and everything, especially what makes a movie a Christmas movie!   You can follow the guys on Instagram @mc.crue.podcast or e-mail us segment ideas at mccruepod@gmail.com  Cheers! "

<div style="background-color: #090e60; padding: 10px; border: 8px solid  white;">
    <p style="text-align: center; font-weight: bold; color: white; font-size: 30px; margin: 0; position: relative; top: 50%; transform: translateY(-50%); font-family: 'Italianno', cursive;">&ndash;&ndash;&ndash;&nbsp;Queries&nbsp;&ndash;&ndash;&ndash;
    </p> 
</div>



In [18]:
urls = [
    "https://trecpodcasts.github.io/resources/podcasts_2020_topics_train.xml",
    "https://trecpodcasts.github.io/resources/podcasts_2020_topics_test.xml"
]

tables = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "xml")
    rows = soup.find_all("topic")

    # Create dict with column headings and empty lists
    table_dict = {"num": [], "query": [], "type": [], "description": []}

    # For every row in the table, find each cell element and add it to the list
    for row in rows:
        table_dict = {
            col: table_dict[col] + [row.find(col).text] for col in table_dict
        }

    table = pd.DataFrame(table_dict)
    table = table.applymap(lambda x: x[0] if isinstance(x, list) else x)
    tables.append(table)

query = pd.concat(tables, ignore_index=True)
query.head()


,num,query,type,description
0,1,coronavirus spread,topical,What were people saying about the spread of th...
1,2,greta thunberg cross atlantic,topical,What were people saying about Greta Thunberg’s...
2,3,black hole image,topical,In May 2019 astronomers released the first-eve...
3,4,story about riding a bird,refinding,I remember hearing a podcast that had a story ...
4,5,daniel ek interview,known item,Someone told me about a podcast interview with...


In [19]:
query = query[~query['num'].isin(['47', '50'])].reset_index(drop=True)
query.head()


,num,query,type,description
0,1,coronavirus spread,topical,What were people saying about the spread of th...
1,2,greta thunberg cross atlantic,topical,What were people saying about Greta Thunberg’s...
2,3,black hole image,topical,In May 2019 astronomers released the first-eve...
3,4,story about riding a bird,refinding,I remember hearing a podcast that had a story ...
4,5,daniel ek interview,known item,Someone told me about a podcast interview with...


In [20]:
pd.options.display.max_colwidth = 200
short_query = query.loc[:, ['num', 'query']].rename(columns={'num': 'qid','query': 'query'})
full_query = query.loc[:, ['num', 'description']].rename(columns={'num': 'qid','description': 'query'})

In [21]:
def remove_non_alphanumeric(row):
    return re.sub(r'[^a-zA-Z0-9 ]+', '', row.query)

full_query['query'] = full_query.apply(remove_non_alphanumeric, axis=1)
full_query.head()

,qid,query
0,1,What were people saying about the spread of the novel coronavirus NCOV19 in Wuhan at the end of 2019
1,2,What were people saying about Greta Thunbergs sailing trip across the Atlantic Ocean in the fall of 2019 and its relationship to global climate change
2,3,In May 2019 astronomers released the firstever picture of a black hole I would like to hear some conversations and educational discussion about the science of astronomy black holes and of the pict...
3,4,I remember hearing a podcast that had a story about a kid riding some kind of bird I want to find it again
4,5,Someone told me about a podcast interview with Daniel Ek CEO of Spotify about the founding and early days of Spotify I would like to find the show and episode that contains that interview Other in...


<div style="background-color: #090e60; padding: 10px; border: 8px solid  white;">
    <p style="text-align: center; font-weight: bold; color: white; font-size: 30px; margin: 0; position: relative; top: 50%; transform: translateY(-50%); font-family: 'Italianno', cursive;">&ndash;&ndash;&ndash;&nbsp;Qrels&nbsp;&ndash;&ndash;&ndash;
    </p> 
</div>




The Relevance assessments (“qrels”) 2021 test set in not available so the 2020 test set has been used to perform it.

In [22]:
url_train = "https://trecpodcasts.github.io/resources/2020_train_qrels.list"
url_test ="https://trecpodcasts.github.io/resources/2020_test_qrels.list"

In [23]:
response = requests.get(url_train)
lines = response.text.split("\n")

data_qrel = []
for line in lines:
    if line:
        query, _, episode_uri, relevance = line.split("\t")
        data_qrel.append({"query": query, "episode_uri": episode_uri, "relevance": relevance})

qrel = pd.DataFrame(data_qrel)
qrel.shape


(609, 3)

In [24]:
# add Podcast Track 2020 test set QRels
url_test = "https://trecpodcasts.github.io/resources/2020_test_qrels.list"
response_test = requests.get(url_test)

lines_test = response_test.text.split("\n")
data_test = []
for line in lines_test:
    if line:
        query, _, episode_uri, relevance = line.split()
        data_test.append({"query": query, "episode_uri": episode_uri, "relevance": relevance})

qrel_test = pd.DataFrame(data_test)
# qrel = qrel.append(qrel_test).reset_index(drop=True)
qrel = pd.concat((qrel, qrel_test)).reset_index(drop=True)
qrel.shape

(10035, 3)

In [25]:
qrel

,query,episode_uri,relevance
0,1,spotify:episode:0E2nqCXMkS218SE72APmNr_240.0,2
1,1,spotify:episode:0E2nqCXMkS218SE72APmNr_300.0,2
2,1,spotify:episode:0E2nqCXMkS218SE72APmNr_360.0,2
3,1,spotify:episode:0Th494DvnO5dU8vTi3QHm2_120.0,1
4,1,spotify:episode:199bOiXL0l4YsRaSNDNXvP_1200.0,2
...,...,...,...
10030,32,spotify:episode:7lrWzeepJQwhr0jyuVEk9m_300.0,0
10031,32,spotify:episode:7lrWzeepJQwhr0jyuVEk9m_600.0,0
10032,32,spotify:episode:7ok6wB1WviyXvxba0wdb5q_720.0,0
10033,32,spotify:episode:7ok6wB1WviyXvxba0wdb5q_780.0,0


In [26]:
qrel['episode_uri'] = qrel['episode_uri'].apply(lambda uri: uri.split('_')[0])
qrel = qrel.groupby(['query', 'episode_uri']).max().reset_index()
qrel

,query,episode_uri,relevance
0,1,spotify:episode:0E2nqCXMkS218SE72APmNr,2
1,1,spotify:episode:0Th494DvnO5dU8vTi3QHm2,1
2,1,spotify:episode:199bOiXL0l4YsRaSNDNXvP,2
3,1,spotify:episode:1ZA1QTtylexrVt75xiprNH,2
4,1,spotify:episode:1o0HCHjNtRWdEWRTWJhzyC,0
...,...,...,...
5825,9,spotify:episode:6v0auT8BbeXzMTmg9FjyDB,1
5826,9,spotify:episode:7EKJswck8jP4tXWHXVJCdO,0
5827,9,spotify:episode:7N0z3FpvsWWTCSgARuCd8C,0
5828,9,spotify:episode:7mIga7ujiMVbbEN5bBQPUX,0


In [37]:
# Find all episodes present in qrels
in_qrels = data.loc[data['episode_uri'].isin(qrel['episode_uri'].unique())]

# Check for NaN and empty string entries
null_or_empty = in_qrels[in_qrels['episode_description'].isna() | (in_qrels['episode_description'] == '')]

# Check for one-word descriptions
def one_word(row):
    return re.search(r'.*\s', row.episode_description)

one_word_descr = list(in_qrels.index[in_qrels.apply(one_word, axis=1).isna()])
one_word_descr


[3972, 101227]

In [40]:
# Find all episodes present in qrels
qrel_uris = set(qrel['episode_uri'].unique())
in_qrels = data[data['episode_uri'].isin(qrel_uris)]

# Check for NaN and empty string entries
null_or_empty = in_qrels[in_qrels['episode_description'].isna() | (in_qrels['episode_description'] == '')]

# Check for one-word descriptions
one_word_descr = in_qrels[in_qrels['episode_description'].str.count(' ') == 0]

# Remove QRels with one-word episode descriptions
qrel = qrel[~qrel['episode_uri'].isin(one_word_descr['episode_uri'])].reset_index(drop=True)

# Find indices of NaN and empty string entries
null_idxs = in_qrels[in_qrels['episode_description'].isna()].index
empty_idxs = in_qrels[in_qrels['episode_description'] == ''].index

# Check for one-word descriptions using a lambda function
one_word_descr = in_qrels[in_qrels.apply(lambda x: len(x['episode_description'].split()) == 1, axis=1)]
one_word_descr.shape

(17, 6)

In [41]:
# Remove QRels with one-word episode descriptions using boolean indexing
qrel = qrel[~qrel['episode_uri'].isin(one_word_descr['episode_uri'])].reset_index(drop=True)
qrel.shape

(5738, 3)

In [42]:
# create dict {episode_uri: doc}
docno_dict = dict(data[['episode_uri', 'docno']].values)

# insert docno for each episode_uri in qrels using dict
qrel['docno'] = qrel['episode_uri'].map(docno_dict)

# rename columns and change data types to make qrels compatible with PyTerrier
qrel = qrel.rename(columns={'query': 'qid', 'relevance': 'label'})
qrel['qid'] = qrel['qid'].astype(str)
qrel['label'] = qrel['label'].astype(int)  # Convert the 'label' column to integers
qrel = qrel[['qid', 'docno', 'label']]

# display qrels
qrel

,qid,docno,label
0,1,3096,2
1,1,6496,1
2,1,15551,2
3,1,21291,2
4,1,24523,0
...,...,...,...
5733,9,93648,1
5734,9,97871,0
5735,9,99768,0
5736,9,102459,0


<div style="background-color: #090e60; padding: 10px; border: 8px solid  white;">
    <p style="text-align: center; font-weight: bold; color: white; font-size: 30px; margin: 0; position: relative; top: 50%; transform: translateY(-50%); font-family: 'Italianno', cursive;">&ndash;&ndash;&ndash;&nbsp;Data&nbsp;&ndash;&ndash;&ndash;
    </p> 
</div>

In [44]:
data.head()

,episode_uri,show_name,show_description,episode_name,episode_description,docno
0,spotify:episode:000A9sRBYdVh66csG2qEdj,Kream in your Koffee,A 20-something blunt female takes on the world and gives you her take on it. Enjoy visits from special guests and friends to give insight and input into interesting situations.,1: It’s Christmas Time!,"On the first ever episode of Kream in your Koffee, Katie talks about tips for Christmas shopping. We also get a little insight into who and what we’ll be hearing about in next weeks episode!",0
1,spotify:episode:000HP8n3hNIfglT2wSI2cA,Morning Cup Of Murder,"Ever wonder what murder took place on today in true crime history? If so, sit back and grab a cup of coffee as you enjoy your daily dose of TC goodness. Your host, Korina Biemesderfer, guides you ...",The Goleta Postal Facility shootings- January 30 2020 - Today in True Crime History,"See something, say something. It’s a mantra many live by. If you see something strange, call it in or make someone aware, even if it seems innocuous. Jennifer San Marco had strange behaviors. It w...",1
2,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Inside The 18 : A Podcast for Goalkeepers by Goalkeepers,"Inside the 18 is your source for all things Goalkeeping! Each week we are joined by guests from around the world. We recap the weeks events, discuss new training techniques and have candid convers...",Ep.36 - Incorporating a Singular Goalkeeping Curriculum in the United States w/ guest Phil Wheddon,"Today’s episode is a sit down Michael and Omar had with former USMNT and former USWNT goalkeeper coach, Phil Wheddon. Phil discusses how he started the International Goalkeeper Coaches Conference ...",2
3,spotify:episode:001i89SvIQgDuuyC53hfBm,Arrowhead Live!,"Your favorite podcast for everything @Chiefs! Providing #ChiefsKingdom with up to the minute news & insight all day, everyday with @KC_Winn & @grant_morse7🎙",Episode 1: Arrowhead Live! Debut,"Join us as we take a look at all current Chiefs news, including the draft, free agency, in-house contract talks, and much more! --- This episode is sponsored by · Anchor: The easiest way to ma...",3
4,spotify:episode:0025RWNwe2lnp6HcnfzwzG,FBoL,"The comedy podcast about toxic characters, writers, and tropes of literature, folklore, myths, and legend. Join host Emily Edwards to discuss feminist literature, toxic masculinity, gender roles...","The Lion, The Witch, And The Wardrobe - Ashley Beall","The modern morality tail of how to stay good for Christmas. Ashley Beall of Make It Modern Podcast (@MIM_Podcast) joins me to discuss the redemption arcs of one of KidLit's biggest fuckbois, Edmun...",4


In [45]:
short_query.head()

,qid,query
0,1,coronavirus spread
1,2,greta thunberg cross atlantic
2,3,black hole image
3,4,story about riding a bird
4,5,daniel ek interview


In [47]:
full_query.head()

,qid,query
0,1,What were people saying about the spread of the novel coronavirus NCOV19 in Wuhan at the end of 2019
1,2,What were people saying about Greta Thunbergs sailing trip across the Atlantic Ocean in the fall of 2019 and its relationship to global climate change
2,3,In May 2019 astronomers released the firstever picture of a black hole I would like to hear some conversations and educational discussion about the science of astronomy black holes and of the pict...
3,4,I remember hearing a podcast that had a story about a kid riding some kind of bird I want to find it again
4,5,Someone told me about a podcast interview with Daniel Ek CEO of Spotify about the founding and early days of Spotify I would like to find the show and episode that contains that interview Other in...


In [48]:
qrel.head()

,qid,docno,label
0,1,3096,2
1,1,6496,1
2,1,15551,2
3,1,21291,2
4,1,24523,0


<div style="background-color: #090e60; padding: 10px; border: 8px solid  white;">
    <p style="text-align: center; font-weight: bold; color: white; font-size: 30px; margin: 0; position: relative; top: 50%; transform: translateY(-50%); font-family: 'Italianno', cursive;">&ndash;&ndash;&ndash;&nbsp;Query Testing and PyTerrier&nbsp;&ndash;&ndash;&ndash;
    </p> 
</div>

In [42]:
# !pip install python-terrier

In [7]:
import os
# os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-20.jdk/Contents/Home"
import pyterrier as pt
import numpy as np
import pandas as pd
if not pt.started():
    pt.init()

# data = pd.read_csv('metadata.tsv', sep = '\t')

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [8]:
#removing na vals as create issue with preprocessing
processed_data = data[data['episode_description'].notna()] 
# data.reset_index(drop = True, inplace = True)
#pre process
processed_data['episode_description'] = processed_data['episode_description'].apply(pre_process)

# data =  data.loc[:10000]
processed_data.head()

,episode_uri,show_name,show_description,episode_name,episode_description,docno
0,spotify:episode:000A9sRBYdVh66csG2qEdj,Kream in your Koffee,A 20-something blunt female takes on the world...,1: It’s Christmas Time!,ever episode kream koffee katie talks tips chr...,0
1,spotify:episode:000HP8n3hNIfglT2wSI2cA,Morning Cup Of Murder,Ever wonder what murder took place on today in...,The Goleta Postal Facility shootings- January ...,see something say something it’s mantra many l...,1
2,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Ep.36 - Incorporating a Singular Goalkeeping C...,today’s episode sit michael omar wi former usm...,2
3,spotify:episode:001i89SvIQgDuuyC53hfBm,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Episode 1: Arrowhead Live! Debut,join us take look current chiefs news includin...,3
4,spotify:episode:0025RWNwe2lnp6HcnfzwzG,FBoL,"The comedy podcast about toxic characters, wri...","The Lion, The Witch, And The Wardrobe - Ashley...",modern morality tail stay good christmas ashle...,4


In [9]:
!rm -rf ./pd_index

#creating a document dictionary to be indexed and stored by pyterrier
docs = processed_data[['docno', 'episode_description']]
docs = docs.rename(columns = {'episode_description': 'text'})
docs = docs.to_dict(orient="records")

pd_indexer = pt.IterDictIndexer("./WebPage/pd_index")
index = pd_indexer.index(docs)

20:47:00.544 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (5) - further warnings are suppressed
20:47:23.549 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer - Indexed 293 empty documents


In [99]:
#below details the info contained
index_file = pt.IndexFactory.of(index)
print(index_file.getCollectionStatistics(), index_file.getMetaIndex())

Number of documents: 105360
Number of terms: 126902
Number of postings: 3917860
Number of fields: 1
Number of tokens: 5211888
Field names: [text]
Positions:   false
 <org.terrier.structures.MetaIndex at 0x12769be50 jclass=org/terrier/structures/MetaIndex jself=<LocalRef obj=0x7f9b5beb0630 at 0x12aca7f10>>


In [100]:
def get_example_index(docid, index_file):
    di = index_file.getDirectIndex()
    doi = index_file.getDocumentIndex()
    lex = index_file.getLexicon()
    example_index = pd.DataFrame(columns=['Term', 'Frequency'])
    
    for posting in di.getPostings(doi.getDocumentEntry(docid)):
        termid = posting.getId()
        lee = lex.getLexiconEntry(termid)
        term_freq = pd.DataFrame([[str(lee.getKey()), str(posting.getFrequency())]], columns=['Term', 'Frequency'])
        example_index = pd.concat([example_index, term_freq])
#         example_index = example_index.append(term_freq)

    example_index = example_index.reset_index(drop=True)
    return example_index

In [101]:
def print_episode_info(docid, metadf):
    show_name = metadf.loc[metadf.docno == str(docid)].show_name.values[0]
    episode_name = metadf.loc[metadf.docno == str(docid)].episode_name.values[0]
    episode_description = metadf.loc[metadf.docno == str(docid)].episode_description.values[0]
    
    print(f'show name: {show_name}')
    print(f'episode name: {episode_name}')
    print(f'episode description: {episode_description}')

In [105]:
docid = 6
example_index = get_example_index(docid, index_file)
print_episode_info(docid, data)

example_index.head()

show name: The Feminization Boudoir
episode name: The Sissy's Mentor (Part 5)
episode description: Miss Jenn Davis reads the final part of The Sissy's Mentor.  It's the story of a young inexperienced college freshman and the dominant woman who takes him under her wing or is that wig?   You can find a lot more great erotic content from Jenn at MissJennDavis.com. Remember that this podcast plays clips (Usually 40%-50%) from audios.  At least half the show is missing and in that half are the very sexiest parts. Please support Candy Apple Press at our clipsites: IWantClips Clips4Sale and buy our books: Amazon.com You can also donate to the Podcast directly at Anchor.fm.  Wherever you listen, please give us good feedback. Thanks, Kylie  ---   This episode is sponsored by  · Anchor: The easiest way to make a podcast.  https://anchor.fm/app  Support this podcast: https://anchor.fm/kylie-gable/support


,Term,Frequency
0,episod,1
1,fm,3
2,http,2
3,show,1
4,support,3


In [116]:
bm25 = pt.BatchRetrieve(index, wmodel='BM25')
pl2 = pt.BatchRetrieve(index, wmodel='PL2')
lgd = pt.BatchRetrieve(index, wmodel='LGD')
dfrbm25 = pt.BatchRetrieve(index, wmodel='DFR_BM25')
dfreeklim = pt.BatchRetrieve(index, wmodel='DFReeKLIM')
bm25f = pt.BatchRetrieve(index, wmodel='BM25F')

model_dict = {'BM25': bm25,
              'PL2': pl2,
              'LGD': lgd,
              'DFRMB25' : dfrbm25,
              'DFREEKLIM' : dfreeklim,
              'BM25F': bm25f}


In [114]:
eval_metrics_list =['map', 'ndcg', 'recip_rank', 'mrt', 'recall']

exp_df_short = pt.Experiment(
    list(model_dict.values()),
    short_query,
    qrel,
    eval_metrics=eval_metrics_list,
    names=list(model_dict.keys())
)
exp_df_short.sort_values(by=['map'], ascending=False)[:5]

,name,map,ndcg,recip_rank,R@5,R@10,R@15,R@20,R@30,R@100,R@200,R@500,R@1000,mrt
4,DFREEKLIM,0.141072,0.297669,0.531197,0.137423,0.168209,0.188210,0.200608,0.220437,0.295078,0.326691,0.361446,0.388210,29.188584
2,LGD,0.133783,0.289684,0.487998,0.134447,0.168885,0.190978,0.199485,0.224030,0.288673,0.328579,0.365176,0.384339,44.410628
3,DFRMB25,0.122409,0.280917,0.494525,0.125483,0.159352,0.177495,0.189535,0.206277,0.282169,0.319773,0.353880,0.373435,41.128918
1,PL2,0.121682,0.278848,0.490628,0.127044,0.150412,0.172392,0.179501,0.194108,0.276917,0.318328,0.351284,0.374561,31.723704
0,BM25,0.121576,0.279072,0.484478,0.127453,0.160993,0.172634,0.187243,0.207548,0.280680,0.319302,0.353728,0.372989,31.639162


In [117]:
exp_df_short = pt.Experiment(
    list(model_dict.values()),
    full_query,
    qrel,
    eval_metrics=eval_metrics_list,
    names=list(model_dict.keys())
)

exp_df_short.sort_values(by=['map'], ascending=False)[:5]

,name,map,ndcg,recip_rank,R@5,R@10,R@15,R@20,R@30,R@100,R@200,R@500,R@1000,mrt
5,BM25F,0.119521,0.273847,0.454871,0.093935,0.141545,0.165461,0.176549,0.194651,0.252839,0.284266,0.339094,0.386635,1018.391599
0,BM25,0.117823,0.270806,0.434107,0.109873,0.155567,0.174526,0.184715,0.199172,0.266976,0.298375,0.345242,0.388832,57.807173
3,DFRMB25,0.116665,0.268631,0.421853,0.110683,0.154856,0.173304,0.187393,0.196851,0.266232,0.298968,0.344494,0.389348,38.743221
4,DFREEKLIM,0.115480,0.263271,0.435876,0.100247,0.128225,0.147711,0.163201,0.186658,0.252630,0.289841,0.343033,0.381409,39.001714
2,LGD,0.110804,0.253801,0.403710,0.104941,0.130880,0.147912,0.168266,0.191016,0.228181,0.274004,0.333968,0.372876,45.988197


In [10]:
#testing query of best fitting model to go into the UI

index_ref = pt.IndexRef.of('./WebPage/pd_index')
# index_1 = index_ref.loadIndex()

searcher = pt.BatchRetrieve(index_ref, wmodel='DFReeKLIM')

# search for a query
query = "game of thrones"
# proc_query = pre_process(query)
results = searcher.search(query)

results

,qid,docid,docno,rank,score,query
0,1,77400,77400,0,19.407843,game of thrones
1,1,53201,53201,1,18.830445,game of thrones
2,1,39711,39711,2,17.912550,game of thrones
3,1,73720,73720,3,17.338374,game of thrones
4,1,62913,62913,4,17.260843,game of thrones
...,...,...,...,...,...,...
995,1,58659,58659,995,5.228509,game of thrones
996,1,75639,75639,996,5.228509,game of thrones
997,1,77514,77514,997,5.228509,game of thrones
998,1,82038,82038,998,5.228509,game of thrones


In [11]:
docno_list = results.loc[:4,'docid'].to_list()
columns = ['show_name', 'episode_name', 'episode_description']

data.loc[data['docno'].isin(docno_list), columns]


,show_name,episode_name,episode_description


In [12]:
docno_list

[77400, 53201, 39711, 73720, 62913]

In [14]:
data.loc[data['docno'].isin(docno_list)]

,episode_uri,show_name,show_description,episode_name,episode_description,docno


In [16]:
metadata = pd.read_pickle('metadata.pkl')
metadata['docno'] = metadata['docno'].astype(int)

In [17]:
metadata.to_pickle('metadata.pkl')